In [9]:
import datetime as dt
import pandas as pd

In [10]:
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet")

In [11]:
df.sample(10)

,origin,price_subtotal,quantity,product_name,invoice_name,client_name,date_invoice,store_name
160364,False,40900.0,1.0,[DAB02788025] DA2788 FILTRO AIRE DONSSON - Ch...,PV6E15204 Mostrador Barranquilla T3/0907,ALVARO PABA TOSCANO,2024-12-12 13:54:03,SUCURSAL BARRANQUILLA
146512,BD1\OUT\56890,532800.0,4.0,[BLS00286125] GS286 FILTRO ACEITE DUAL MOTORES...,FCOT22467,MARIA ESPERANZA HERNANDEZ OSPINA,2025-01-15 16:55:47,PRINCIPAL COTA
4865,BD5\OUT\63410,212000.0,4.0,[DAS07805025] DA7805 FILTRO AIRE SELLADO PLANT...,FMDE3936,AGROFILTER ANTIOQUIA S.A.S,2025-09-20 16:28:45,SUCURSAL MEDELLIN
161767,BD9/OUT/03704,78000.0,2.0,"[BLS00278125] GS278 FILTRO ACEITE CHEVROLET,JA...",FCAL3124,SERVICIOS AGROMECANICOS DE OCCIDENTE S.A.S. SE...,2024-12-10 15:56:58,SUCURSAL CALI
43172,False,120000.0,1.0,[BLS00141125] GS141 FILTRO ACEITE DUAL CUMMINS...,PV9E1387 Mostrador Cali T1/1747,SOLUCIONES TECNICAS DIESEL S.A.S,2025-07-16 18:10:23,SUCURSAL CALI
177507,BD42/OUT/00104,24600.0,1.0,[BCS00063125] GS063 FILTRO COMBUSTIBLE 2o. DET...,FCOT21627,RH GROUP SAS,2024-11-12 20:00:23,PRINCIPAL COTA
74900,BD41/OUT/08016,27000.0,1.0,[DCE00764189] G764 FILTRO SEPARDOR COMBUSTIBLE...,FCAL4433,MARIO ALBERTO HUERTAS COTES,2025-05-19 21:21:39,SUCURSAL CALI
42185,BD1\OUT\59310,180400.0,4.0,"[BLS00037125] GS037 FILTRO ACEITE MACK, CATERP...",FCOT25348,SERGIO DAVID CORTES ORJUELA,2025-07-17 20:11:48,PRINCIPAL COTA
58045,False,39800.0,1.0,[DAR03066025] DA3066 FILTRO AIRE 1o. A.CHALLME...,PV2E40801 Mostrador Calle 6 T1/24426,MILLER DANIEL PINTO CARVAJAL,2025-06-18 20:22:30,SUCURSAL CALLE 6
93661,False,23300.0,1.0,[DAB04772025] DA4772 FILTRO AIRE INTERNO BOBCA...,PV4E14736 Mostrador Norte T1/17113,FAROCOR TRANSPORTES SAS,2025-04-14 20:29:28,SUCURSAL NORTE


In [12]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])

df = df.drop(columns=["origin"])

df["product_ref"] = df["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df["date_invoice"] = pd.to_datetime(df["date_invoice"])
df["mes"] = df["date_invoice"].dt.month
df["año"] = df["date_invoice"].dt.year
df["dia"] = df["date_invoice"].dt.day

In [13]:
df = df[~(df["product_ref"].isna())]

In [17]:
from datetime import datetime

# Fecha y hora actual en formato YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d")

# Nombre dinámico
filename = f"/home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_{timestamp}.xlsx"

# Guardar el Excel
df.to_excel(filename, index=False)

print(f"Archivo guardado: {filename}")

#Tiempo promedio 1.5 minutos

Archivo guardado: /home/donsson/proyectos/VENTAS/prom_ventas/promedio_ventas_20250929.xlsx


In [16]:
df.head()

,price_subtotal,quantity,product_name,invoice_name,client_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega,product_ref,mes,año,dia
0,208500.0,3.0,[BCE00858125] G858 FILTRO COMBUSTIBLE CAMIONES...,FCAL5704,FILTROS Y FILTROS S.A.S,2025-09-29 17:25:53,SUCURSAL CALI,VENDEDOR EXTERNO,BD9,SUCURSAL CALI,BCE00858125,9,2025,29
1,306800.0,2.0,"[DAB02768025] DA2768 FILTRO AIRE KENWORTH, PET...",PV4E16297 Mostrador Norte T2/13117,MARIA JOSEFA FORERO,2025-09-29 17:23:54,SUCURSAL NORTE,MOSTRADOR,<NA>,SUCURSAL NORTE,DAB02768025,9,2025,29
2,206200.0,1.0,[BCS00708125] GS708 FILTRO COMBUSTIBLE KENWOR...,PV4E16297 Mostrador Norte T2/13117,MARIA JOSEFA FORERO,2025-09-29 17:23:54,SUCURSAL NORTE,MOSTRADOR,<NA>,SUCURSAL NORTE,BCS00708125,9,2025,29
3,74800.0,1.0,[DAR09096025] DA9096 FILTRO AIRE SEGURIDAD KOB...,PV2E43732 Mostrador Calle 6 T1/25541,HIDRAULICOS ARAUCA S.A.S. ZOMAC,2025-09-29 17:18:49,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6,DAR09096025,9,2025,29
4,84900.0,1.0,[DAB08191025] DA8191 FILTRO AIRE FOTON AUMARK ...,PV2E43732 Mostrador Calle 6 T1/25541,HIDRAULICOS ARAUCA S.A.S. ZOMAC,2025-09-29 17:18:49,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6,DAB08191025,9,2025,29


In [ ]:
fecha_ini = "2025-01-01"
fecha_fin = "2025-06-30"

df["date_invoice"] = pd.to_datetime(df["date_invoice"])


df_final = df[(df["date_invoice"] >= fecha_ini) & (df["date_invoice"] <= fecha_fin)]


In [30]:
df_final["date_invoice"].min()

Timestamp('2025-01-02 12:33:39')

In [31]:
df_final["date_invoice"].max()

Timestamp('2025-06-28 18:02:21')

In [35]:
df_agru = df_final.groupby("client_name").agg({"price_subtotal":"sum",
                                               "invoice_name":"count"}).reset_index()

In [41]:
clientes_ene_jun  = df_agru[(df_agru["invoice_name"]>=4) & (df_agru["price_subtotal"]> 1400000 )]

clientes_ene_jun = clientes_ene_jun.rename(columns={"invoice_name":"cantidad de facturas"})

clientes_ene_jun.to_excel("ventasmayoresalminimodesdeenerohastajunio2025.xlsx")